In [90]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [91]:
train_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_transforms = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [92]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transforms)

Files already downloaded and verified


In [93]:
valid_size = 0.2

num_train = len(trainset)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [98]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,sampler=train_sampler,
                                         num_workers=2)

validloader = torch.utils.data.DataLoader(trainset, batch_size=4, sampler=valid_sampler, num_workers=2)

In [5]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=test_transforms)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                          shuffle=True, num_workers=2)

Files already downloaded and verified


In [6]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [136]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self, num_classes=10):
          super(Net, self).__init__()
          self.layer1 = nn.Sequential(
              nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
              nn.BatchNorm2d(16),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2, stride=2))
          self.layer2 = nn.Sequential(
              nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
              nn.BatchNorm2d(32),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2, stride=2))
          self.fc1 = nn.Linear(8*8*32, 64)
          self.fc2 = nn.Linear(64, 128) #num_classes
          self.fc3 = nn.Linear(128, num_classes)
          self.softmax = nn.LogSoftmax(dim=1)
          self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
          out = self.dropout(self.layer1(x))
          out = self.dropout(self.layer2(out))
          out = out.reshape(out.size(0), -1)
          out = self.dropout(self.fc1(out))
          out = self.dropout(self.fc2(out))
          out = self.fc3(out)
          return self.softmax(out)



def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

net = Net().apply(init_weights).to(device)


In [137]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)

In [138]:
from tqdm import tqdm

epochs = 10
valid_loss_min = np.Inf
total_step = len(trainloader)


for epoch in range(epochs):
  train_loss = 0.0
  valid_loss = 0.0

  net.train()
  for i,data in enumerate(trainloader):
    inputs,labels = data[0].to(device), data[1].to(device)

    output = net(inputs)
    loss = criterion(output,labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss += loss.item()*inputs.size(0)
    

  net.eval()
  for i,data in enumerate(validloader):
     data,target = data[0].to(device), data[1].to(device)
     # forward pass: compute predicted outputs by passing inputs to the model
     output = net(data)
     # calculate the batch loss
     loss = criterion(output, target)
     # update average validation loss 
     valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
  train_loss = train_loss/len(trainloader.sampler)
  valid_loss = valid_loss/len(validloader.sampler)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss, valid_loss))
  
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
      print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
      torch.save(net.state_dict(), './cifar_net.pth')
      valid_loss_min = valid_loss  

Epoch: 1 	Training Loss: 1.694843 	Validation Loss: 1.327869
Validation loss decreased (inf --> 1.327869).  Saving model ...
Epoch: 2 	Training Loss: 1.333359 	Validation Loss: 1.241535
Validation loss decreased (1.327869 --> 1.241535).  Saving model ...
Epoch: 3 	Training Loss: 1.233050 	Validation Loss: 1.128666
Validation loss decreased (1.241535 --> 1.128666).  Saving model ...
Epoch: 4 	Training Loss: 1.171696 	Validation Loss: 1.042166
Validation loss decreased (1.128666 --> 1.042166).  Saving model ...
Epoch: 5 	Training Loss: 1.129386 	Validation Loss: 1.053908
Epoch: 6 	Training Loss: 1.102801 	Validation Loss: 1.045650
Epoch: 7 	Training Loss: 1.080632 	Validation Loss: 1.006529
Validation loss decreased (1.042166 --> 1.006529).  Saving model ...
Epoch: 8 	Training Loss: 1.053985 	Validation Loss: 0.974927
Validation loss decreased (1.006529 --> 0.974927).  Saving model ...
Epoch: 9 	Training Loss: 1.046851 	Validation Loss: 1.012638
Epoch: 10 	Training Loss: 1.036176 	Valida

In [139]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

# **Prediction time**

In [140]:
net = Net().to(device)
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [141]:
correct=0
total=0

net.eval()
with torch.no_grad():
  for data in testloader:
    images,labels = data[0].to(device), data[1].to(device)
    
    outputs = net(images)
    _ , predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (predicted==labels).sum().item()

  print('Accuracy of the network on the 10000 test images: {}'.format(correct*100/total))


Accuracy of the network on the 10000 test images: 66.88
